In [2]:
import pymongo
from pymongo import MongoClient

import random
from cassiopeia import riotapi

from collections import deque

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.print_calls(True)
summoner = riotapi.get_summoner_by_name("smilming")
print "Name:" + summoner.name
print "Level:" + str(summoner.level)


https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/by-name/smilming?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Name:smilming
Level:30


In [3]:
masters = riotapi.get_master()

https://oce.api.pvp.net/api/lol/oce/v2.5/league/master?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24&type=RANKED_SOLO_5x5
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/555008,1132033,229379,707158,627719,314888,1260460,4461579,5540365,497679,7600144,638484,498717,367137,319522,1380903,372999,232626,5540914,1343028,3762231,1530424,578105,4560956,2573886,3262016,380994,303286,632903,620617,724554,297036,6381137,554066,1153622,3210839,4890200,579674,229981,285278?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/397919,5530208,2922593,1505382,220270,399473,890994,402017,399995,222336,276609,3520130,774275,1363078,508040,392266,281749,323736,289945,2910362,6070946,620195,276134,329383,1321671,1160364,402607,324274,221555,6851253,402614,323773,272065,915141,431814,447687,632520,498210,272845,365776?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/319185,4802261,302301,369886,293793,3302634,294124,2

In [61]:
print masters[0].division
print masters[0].league_points
test_stats = masters[0].summoner.stats()

Division.one
231
https://oce.api.pvp.net/api/lol/oce/v1.3/stats/by-summoner/372999/summary?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24


In [82]:
print masters.tier

Tier.master


In [73]:
print test_stats[cassiopeia.type.core.common.StatSummaryType.ranked_solo].to_json()

{
    "aggregatedStats": {
        "averageAssists": 0, 
        "averageChampionsKilled": 0, 
        "averageCombatPlayerScore": 0, 
        "averageNodeCapture": 0, 
        "averageNodeCaptureAssist": 0, 
        "averageNodeNeutralize": 0, 
        "averageNodeNeutralizeAssist": 0, 
        "averageNumDeaths": 0, 
        "averageObjectivePlayerScore": 0, 
        "averageTeamObjective": 0, 
        "averageTotalPlayerScore": 0, 
        "botGamesPlayed": 0, 
        "killingSpree": 0, 
        "maxAssists": 0, 
        "maxChampionsKilled": 0, 
        "maxCombatPlayerScore": 0, 
        "maxLargestCriticalStrike": 0, 
        "maxLargestKillingSpree": 0, 
        "maxNodeCapture": 0, 
        "maxNodeCaptureAssist": 0, 
        "maxNodeNeutralize": 0, 
        "maxNodeNeutralizeAssist": 0, 
        "maxNumDeaths": 0, 
        "maxObjectivePlayerScore": 0, 
        "maxTeamObjective": 0, 
        "maxTimePlayed": 0, 
        "maxTimeSpentLiving": 0, 
        "maxTotalPlayerScore"

In [5]:
unpulled_summoners = deque(entry.summoner for entry in masters)
pulled_summoners = deque()

In [7]:
client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches

In [52]:
test_summoner = masters[0].summoner
print test_summoner.name
print test_summoner.id
print test_summoner.data.summonerLevel
print test_summoner.data.revisionDate
print masters[0].to_json()

cat and bird
372999
30
1479534704000
{
    "division": "I", 
    "isFreshBlood": true, 
    "isHotStreak": false, 
    "isInactive": false, 
    "isVeteran": false, 
    "leaguePoints": 231, 
    "losses": 174, 
    "miniSeries": null, 
    "playerOrTeamId": "372999", 
    "playerOrTeamName": "cat and bird", 
    "wins": 226
}


In [35]:
summoner_dict = dict()
summoner_dict['name'] = test_summoner.name
summoner_dict['id'] = test_summoner.id
summoner_dict['summonerLevel'] = test_summoner.data.summonerLevel
summoner_dict['revisionDate'] = test_summoner.data.revisionDate

In [36]:
print summoner_dict

{'summonerLevel': 30, 'revisionDate': 1479534704000, 'name': u'cat and bird', 'id': 372999}


In [47]:
summoners_collection.insert_one(summoner_dict)

In [48]:
#show all entries
for entry in summoners_collection.find():
    print entry

{u'id': 372999, u'_id': ObjectId('583502eb666d59386fcd9192'), u'revisionDate': 1479534704000L, u'name': u'cat and bird', u'summonerLevel': 30}


In [49]:
#delete all entries
summoners_collection.delete_many({})